In [191]:
import importlib

import plotly.graph_objects as go


In [192]:
# Manually reloading python module such that
# jupyter reflects changes without kernel restart

import apollo.api.yahoo_api_connector as yac
import apollo.calculations.average_true_range as atr
import apollo.calculations.distribution_moments as dm
from apollo.utils.common import to_default_date_string

importlib.reload(yac)
importlib.reload(atr)
importlib.reload(dm);

In [193]:
ticker = "SPY"
start_date = "2024-01-01"
end_date = "2024-04-01"

api_connector = yac.YahooApiConnector(ticker, start_date, end_date)
dataframe = api_connector.request_or_read_prices()

dataframe;

In [194]:
WINDOW_SIZE = 5
KURTOSIS_THRESHOLD = 0.0
VOLATILITY_MULTIPLIER = 1.0

dm_calculator = dm.DistributionMomentsCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)

dm_calculator.calculate_distribution_moments()

atr_calculator = atr.AverageTrueRangeCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)

atr_calculator.calculate_average_true_range()

dataframe.dropna(inplace=True)

# Long Mean Reversion:
# Kurtosis > 0
# Skew < 0
# Vol > 0

# Short Mean Reversion:
# Kurtosis < 0
# Skew > 0
# Vol < 0

dataframe["signal"] = 0

long = (
    (dataframe["kurt"] > KURTOSIS_THRESHOLD) &
    (dataframe["skew"] < 0) &
    (dataframe["tr"] > dataframe["atr"] * VOLATILITY_MULTIPLIER)
)
dataframe.loc[long, "signal"] = 1

short = (
    (dataframe["kurt"] < KURTOSIS_THRESHOLD) &
    (dataframe["skew"] > 0) &
    (dataframe["tr"] > dataframe["atr"] * VOLATILITY_MULTIPLIER)
)
dataframe.loc[short, "signal"] = -1

dataframe

,ticker,open,high,low,close,volume,avg,std,skew,kurt,z_score,tr,atr,signal
date,,,,,,,,,,,,,,
2024-01-12,SPY,477.839996,478.600006,475.230011,476.679993,57944000,475.614001,1.285290,-0.745642,-2.282144,0.829378,3.369995,5.233992,0
2024-01-16,SPY,475.260010,476.609985,473.059998,474.929993,85014900,475.679999,1.227373,-0.962608,-1.051478,-0.611066,3.619995,4.911193,0
2024-01-17,SPY,471.820007,472.790009,469.869995,472.290009,68843900,475.362000,1.855547,-1.571648,2.030306,-1.655572,5.059998,4.940954,1
2024-01-18,SPY,474.010010,477.059998,472.420013,476.489990,91856200,475.347998,1.844479,-1.581024,2.079884,0.619141,4.769989,4.906761,0
2024-01-19,SPY,477.649994,482.720001,476.540009,482.429993,110733300,476.563995,3.720409,0.966212,1.948905,1.576707,6.230011,5.171411,0
2024-01-22,SPY,484.010010,485.220001,482.779999,483.450012,75844900,477.917999,4.837430,0.196190,-2.434437,1.143585,2.790009,4.695130,0
2024-01-23,SPY,484.010010,485.109985,482.890015,484.859985,49945300,479.903998,5.318481,-0.827469,-1.284052,0.931843,2.219971,4.200098,0
2024-01-24,SPY,487.809998,488.769989,484.880005,485.390015,81765000,482.523999,3.568840,-1.673927,2.969705,0.803066,3.910004,4.142079,0
2024-01-25,SPY,487.579987,488.309998,485.390015,488.029999,72525000,484.832001,2.134180,0.702209,0.533712,1.498467,2.919983,3.897660,0


In [195]:
prime_value = "close"
trace_value = "signal"

x = dataframe.index.to_numpy()

y1 = dataframe[prime_value].to_numpy()
y2 = dataframe[trace_value].to_numpy()

# Create the first trace with the primary y-axis
trace1 = go.Scatter(x=x, y=y1, name=prime_value)

# Create the second trace with the secondary y-axis
trace2 = go.Scatter(x=x, y=y2, name=f"{trace_value}", yaxis="y2")

# Plot title
title = (
    f"{ticker}:"
    f" {to_default_date_string(dataframe.index.to_numpy()[0])}"
    f" - {to_default_date_string(dataframe.index.to_numpy()[-1])}"
)

# Create the layout with two y-axes
layout = go.Layout(
    title=title,
    yaxis={},
    yaxis2={"overlaying": "y", "side": "right"},
    height=650,
)

# Create the figure and add traces to it
fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.update_xaxes(
    showspikes=True,
    spikemode="across",
    spikecolor="black",
    spikethickness=0.5,
)